In [1]:
#file used to generate outputs for presentations
import warnings
#use this when printing the output to a pdf
warnings.filterwarnings('ignore')
#use this when doing coding normally
#warnings.filterwarnings('default')


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

sae = pd.read_csv('sae.csv')
#sae.head()

Melchors work merging hours and cases

In [ ]:


"""# Cases


* Change date to date_time object
    * Create a new Date column for both datasets
    * Allows merging on shared column
*  Sort dates in ascending order


"""

cases = pd.read_csv("Cases.csv")
cases

# check type, length
cases.CLNDR_DT

#change CLNDR_DT from object to date_time
  # make sure to create new column 'Date' for both datasets so they share common 
  # for merging

cases['Date'] = pd.to_datetime(cases['CLNDR_DT'])
cases

#sort dates from past to current
cases = cases.sort_values(by='CLNDR_DT')
cases

#drop redundant columns

#^Dont need this line
#cases = cases.drop(columns=['CLNDR_DT', 'DIV_NM'])
#cases

# check number of NaN values
cases.isna().sum()

## needed to drop NaN in order for 'tot_each_day' to be correct number
    ## ask why including NaN would yield wrong sum
        ## what values does NaN have?

cases = cases.dropna()
cases

## create new dataframe from cases
    ## focusing on getting total of eaches selected per day
cases_tot_day = pd.DataFrame(cases)

## group by Date and BRNCH_CD, sum eaches_selected by each zone into one total each column
cases_tot_day['Total_Each_Day'] = pd.DataFrame(cases.groupby(['Date', 'BRNCH_CD'])['EACHES_SELECTED'].transform('sum'))
cases_tot_day

## New Dataframe with order = Date, Branch, zone, eaches selected, total eaches
cases4 = pd.DataFrame(cases, columns = (['Date', 'BRNCH_CD', 'ZONE', 'EACHES_SELECTED', 'Total_Each_Day']))

## Change index starting at 1, drop existing index to avoid adding as column
cases4 = cases4.reset_index(drop=True)

cases4.BRNCH_CD[0]

## get ratio of total eaches per zone
    ## eaches selected of zone / total eaches
    ## add ratio to a list and append to dataset after

ratio = []
for i in range(0, len(cases4)):
  #print(i)
  r = cases4.EACHES_SELECTED[i] / cases4.Total_Each_Day[i]
  ratio.append(r)

ratio[0:10]

## add list of ratios to dataframe, should be in order
cases4['ratio'] = ratio

#type(cases4.ZONE[1])

## create a list for every zone (dry, clr, frz)
## for every row in dataframe:
    ## if the zone of the row == "DRY":
        ## add the ratio at index i to the "DRY" list
        ## add zeros to the other lists (ClR, FRZ) (place holder)
    ## if zone of row == "CLR":
        ## add ratio to CLR list
        ## add zeros to other lists
    ## repeat for FRZ

## add lists as columns to dataset

dry_ratio = []
clr_ratio = []
frz_ratio = []

for i in range(0, len(cases4)): 
  if cases4.ZONE[i] == 'DRY':
    dry_ratio.append(cases4.ratio[i])
    clr_ratio.append(0)
    frz_ratio.append(0)
  elif cases4.ZONE[i] == 'FRZ':
    dry_ratio.append(0)
    clr_ratio.append(0)
    frz_ratio.append(cases4.ratio[i])
  elif cases4.ZONE[i] == 'CLR':
    dry_ratio.append(0)
    clr_ratio.append(cases4.ratio[i])
    frz_ratio.append(0)


cases4['dry_ratio'] = dry_ratio
cases4['clr_ratio'] = clr_ratio
cases4['frz_ratio'] = frz_ratio

## sort dataframe by date and branch to see everything together
    ## check if math is correct
cases4 = cases4.sort_values(by=['Date', 'BRNCH_CD'], ascending = [True, True])

## smoosh multiple rows of same branch and date into 1 row
    ## aggregate sums, zeros as placeholders will have no effect on ratio
cases6 = cases4.groupby(['Date', 'BRNCH_CD', 'Total_Each_Day'], as_index = False).agg('sum')


## drop eaches since same as total
## drop ratio since all ratios add up to 1 anyway
cases_EachPerDay = cases6.drop(columns=['EACHES_SELECTED', 'ratio'])

"""# Hours


*   Repeat steps from cases


"""

hours = pd.read_csv("hours.csv")

hours.REPORT_DATE

hours['Date'] = pd.to_datetime(hours['REPORT_DATE'])

hours = hours.sort_values(by=['REPORT_DATE', 'BRNCH_CD'], ascending = [True, True])

hours = hours.drop(columns=['DIV_NBR', 'REPORT_DATE', 'FULL_MARKET_NAME'])

## drop "NaN" (UNKOWN)
hours = hours[hours.COHORT != 'UNKNOWN']

## create datarame in order desired
hours_co = pd.DataFrame(hours, columns = ['Date', 'BRNCH_CD', 'COHORT', 'CASES_SELECTED', 'MEASURED_DIRECT_HRS'])

## group by Date and branch to get sum of HOURS MEASURED of each cohort as total hours worked at branch on specific day
hours_co['Total_Hours'] = pd.DataFrame(hours.groupby(['Date', 'BRNCH_CD'])['MEASURED_DIRECT_HRS'].transform('sum'))


## group by Date and branch to get sum of CASES SELECTED of each cohort as total cases selected at branch on specific day
hours_co['Total_Cases'] = pd.DataFrame(hours.groupby(['Date', 'BRNCH_CD'])['CASES_SELECTED'].transform('sum'))


hours_co = hours_co.reset_index(drop = True)

## like hours, get ratio of hours worked per cohort relative to total hours measured at branch on specific day
Hrs_PerCo = []
for i in range (0, len(hours_co)):
  r = hours_co.MEASURED_DIRECT_HRS[i] / hours_co.Total_Hours[i]
  Hrs_PerCo.append(r)

Hrs_PerCo[0:5]

hours_co['Hrs_Pct'] = Hrs_PerCo
hours_co

## same process of placing respective hours ratios per cohort as eaches per zone in cases
    ## 0.000 = placeholder for aggregation
A_Hours = []
B_Hours = []
C_Hours = []

for i in range(0, len(hours_co)):
  if hours_co.COHORT[i] == 'A':
    A_Hours.append(hours_co.Hrs_Pct[i])
    B_Hours.append(0)
    C_Hours.append(0)
  elif hours_co.COHORT[i] == 'C':
    A_Hours.append(0)
    B_Hours.append(0)
    C_Hours.append(hours_co.Hrs_Pct[i])
  elif hours_co.COHORT[i] == 'B':
    A_Hours.append(0)
    B_Hours.append(hours_co.Hrs_Pct[i])
    C_Hours.append(0)


hours_co['A_HrsPct'] = A_Hours
hours_co['B_HrsPct'] = B_Hours
hours_co['C_HrsPct'] = C_Hours

hours_co

## ratio of cases per cohort
Cases_PerCo = []
for i in range (0, len(hours_co)):
  r = hours_co.CASES_SELECTED[i] / hours_co.Total_Cases[i]
  Cases_PerCo.append(r)

hours_co['Cases_Pct'] = Cases_PerCo
hours_co

## cases ratio per cohort w/ placeholders
A_Cases = []
B_Cases = []
C_Cases = []

for i in range(0, len(hours_co)):
  if hours_co.COHORT[i] == 'A':
    A_Cases.append(hours_co.Cases_Pct[i])
    B_Cases.append(0)
    C_Cases.append(0)
  elif hours_co.COHORT[i] == 'C':
    A_Cases.append(0)
    B_Cases.append(0)
    C_Cases.append(hours_co.Cases_Pct[i])
  elif hours_co.COHORT[i] == 'B':
    A_Cases.append(0)
    B_Cases.append(hours_co.Cases_Pct[i])
    C_Cases.append(0)


hours_co['A_Cases'] = A_Cases
hours_co['B_Cases'] = B_Cases
hours_co['C_Cases'] = C_Cases

hours_co

hours_CoHrs = hours_co.groupby(['Date', 'BRNCH_CD', 'Total_Hours', 'Total_Cases'], as_index = False).agg('sum')
hours_CoHrs

"""# New Datasets"""

cases_EachPerDay

## Date = CLNDR_DATE
## BRNCH_CD = code referring to sepcific Warehouse

## Total_Each_Day = sum of eaches selected by all zones on specific date
    ## dry_ratio = proportion of dry products relative to total eaches selected on specific date 
    ## clr_ratio = proportion of cooler products relative to total eaches selected on specific date
    ## frz_ratio = proportion of freezer products relative to total eaches selected on specific date

hours_CohortHrsCases = pd.DataFrame(hours_CoHrs, columns = ['Date', 'BRNCH_CD', 
                                                            'Total_Hours', 'Total_Cases',
                                                            'A_HrsPct',
                                                            'B_HrsPct', 'C_HrsPct', 
                                                            'A_Cases', 'B_Cases','C_Cases' 
                                                            ])
hours_CohortHrsCases

## Date = REPORT_DATE
## BRNCH_CD = code referring to sepcific Warehouse

## Total_Hours = sum of hours worked by all cohorts on specific date
    ## A_HrsPct = proportion of hours worked by Cohort A relative to total hours on specific date
    ## B_HrsPct = proportion of hours worked by Cohort B relative to total hours on specific date
    ## C_HrsPct = proportion of hours worked by Cohort C relative to total hours on specific date

## Total_Cases = Sum of all cases selected by all cohorts on specific date
    ## A_Cases = proportion of cases selected by Cohort A relative to total cases selected on specific date
    ## B_Cases = proportion of cases selected by Cohort B relative to total cases selected on specific date
    ## C_Cases = proportion of cases selected by Cohort C relative to total cases selected on specific date

"""# Merge
* lost a lot of data :/
* dont merge yet?
"""

Foods = hours_CohortHrsCases.merge(cases_EachPerDay, how = 'inner', on = ['Date', 'BRNCH_CD'])


My work adding dates to the dataset and merging Melchor's dataset with the sae dataset. I also created dummy variables for the categorical variables (month, weekday, branch_cd)

In [5]:
dummies = False #boolean which determiens whether to drop the dummy variables or not
Foods2 = Foods

#adding the response variable(s) to the dataset
Foods2["cpmh"] = Foods2['Total_Cases']/Foods2['Total_Hours'] #creates the dependent variable we are trying to measure
Foods2.replace([np.inf, -np.inf], np.nan, inplace=True) #converting all infinites to na's. will want to come up with a better solution later 
Foods2 = Foods2.dropna() #drops all na's for now

""" Will probably not want to use this as eaches is not connected to the hours variable
epmh_bool = True #set this to true if we want to use epmh as a response as well
if(epmh_bool):
  Foods2["epmh"] = Foods2['Total_Each_Day']/Foods2['Total_Hours'] #creates the dependent variable we are trying to measure
  Foods2.replace([np.inf, -np.inf], np.nan, inplace=True) #converting all infinites to na's. will want to come up with a better solution later 
  Foods2 = Foods2.dropna() #drops all na's for now
"""


weekdays = {0: "Monday",
            1: "Tuesday",
            2: "Wednesday",
            3: "Thursday",
            4: "Friday",
            5: "Saturday",
            6: "Sunday"}

months =   {1: "January",
            2: "February",
            3: "March",
            4: "April",
            5: "May",
            6: "June",
            7: "July",
            8: "August",
            9: "September",
            10: "October",
            11: "November",
            12: "December"}
            
Foods2['weekday'] = Foods2['Date'].apply(lambda x: weekdays[x.weekday()]) #adds the weekday of the particular entry 
Foods2['month'] = Foods2['Date'].apply(lambda x: months[x.month]) #add the reporting_date month as a separate variable


#adds the sae data to the main dataset
sae = pd.read_csv('sae.csv')

Foods3 = Foods2.merge(sae, how='outer', on='BRNCH_CD')



#to add dummy variables to the dataset 
if(dummies):
  Foods3 = pd.concat([Foods2, pd.get_dummies(Foods2['BRNCH_CD'])], axis=1)
  Foods3 = pd.concat([Foods2, pd.get_dummies(Foods2['weekday'])], axis=1)
  Foods3 = pd.concat([Foods2, pd.get_dummies(Foods2['month'])], axis=1)
  Foods3 = Foods2.drop(["BRNCH_CD", 'Date', 'month', 'weekday'], axis=1)




#Foods3.to_csv('model_data.csv', index=False)
Foods3.head()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,BRNCH_CD,Total_Hours,Total_Cases,A_HrsPct,B_HrsPct,C_HrsPct,A_Cases,B_Cases,C_Cases,Total_Each_Day,dry_ratio,clr_ratio,frz_ratio,cpmh,weekday,month,GO_LIVE_DATE,LABEL_TYPE
0,2018-12-31,2Z,1.155000,50,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,44059,0.393631,0.302799,0.303570,43.290043,Monday,December,2019-05-05,MOBILE TEAR
1,2019-01-02,2Z,197.701943,35071,0.863320,0.071466,0.065215,0.857461,0.075048,0.067492,40948,0.455920,0.258889,0.285191,177.393300,Wednesday,January,2019-05-05,MOBILE TEAR
2,2019-01-03,2Z,262.539447,45072,0.785586,0.153591,0.060824,0.789781,0.144058,0.066161,35093,0.420169,0.286382,0.293449,171.677058,Thursday,January,2019-05-05,MOBILE TEAR
3,2019-01-04,2Z,1.940000,42,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,46007,0.442454,0.271133,0.286413,21.649485,Friday,January,2019-05-05,MOBILE TEAR
4,2019-01-07,2Z,211.180832,35696,0.820379,0.148810,0.030811,0.807485,0.165985,0.026530,34949,0.397579,0.304844,0.297576,169.030492,Monday,January,2019-05-05,MOBILE TEAR


Cross-validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

foods4 = pd.read_csv("model_data.csv")
foods4 = foods4.drop(columns=['Total_Cases', 'Total_Hours', 'C_HrsPct', 'C_Cases', 'frz_ratio'])
foods4 = foods4.drop(columns=['GO_LIVE_DATE']) #dropping go_live_date for now

foods4.columns

#converting variables to work with the sklearn model 
foods4['Date'] = pd.to_datetime(foods4['Date']).astype(int)


foods4 = pd.concat([foods4, pd.get_dummies(foods4['BRNCH_CD'])], axis=1)
foods4 = pd.concat([foods4, pd.get_dummies(foods4['weekday'])], axis=1)
foods4 = pd.concat([foods4, pd.get_dummies(foods4['month'])], axis=1)
foods4 = pd.concat([foods4, pd.get_dummies(foods4['LABEL_TYPE'])], axis=1)
foods4 = foods4.drop(["BRNCH_CD", 'Date', 'month', 'weekday', 'LABEL_TYPE'], axis=1)


#splitting the data up for cross validation
x = foods4['cpmh'].values
y = foods4.drop(columns='cpmh').values


x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3) 

#scores = cross_val_score() #can use this function to score our cross validation

#converting the data to create a model and then creating the model
x_train = np.array(x_train).reshape(-1,1)

reg = LinearRegression().fit(x_train, y_train)

x_test = np.array(x_test).reshape(-1,1)
reg.score(x_test, y_test)

x = np.array(x).reshape(-1, 1)
cross_val_score(reg, x,y) #finding r^2 scores using cross-validation


My work seeing if adding weather data helps

In [ ]:
from meteostat import Daily, Stations
from geopy import Nominatim
from tqdm import tqdm
from datetime import timedelta

foods_w = Foods3[Foods3['cpmh'] < 300] #dropping all cpmh over 300 just for this analysis


locator = Nominatim(user_agent='myGeocoder')
#loc = locator.geocode('PITTSTON SYSTEMS')


div_nm = {} #dictionary that stores the division name and its cooresponding branch code to easily add weather data later on
for n in cases['DIV_NM'].unique():
    div_nm[n] = cases.where(cases['DIV_NM'] == n)['BRNCH_CD'].dropna(how='all').iloc[0]

#create a dictionary which stores the info from div_nm in a more useful way to create a dataframe
div = {'name': div_nm.keys(), 'BRNCH_CD':div_nm.values()}
div_df = pd.DataFrame(data=div)

#get the location data for each warehouse
loc = []
for d in tqdm(div_nm):
    loc.append(locator.geocode(d))

loc1 = []
for l in loc:
    try:
        loc1.append(l[1])
    except:
        loc1.append(0)

div_df['location'] = loc1 

#drop the division name and then merge the two databases together 
div_df = div_df.drop('name', axis=1)
foods_w = foods_w.merge(div_df, how='inner', on='BRNCH_CD')

#get the station info for each location and get the weather for the day in question
station = Stations()
stat = {}
for l in foods_w['location'].unique():
    try:
        stat[l] = station.nearby(l[0], l[1]).fetch(1)
    except:
        stat[l] = 0 #assigns a 0 for each location that was unable to be determined prior
        

foods_w = foods_w[foods_w['location'] != 0] #drops rows where the location is equal to 0 

it = foods_w.itertuples(index=False)
weather = []
num = 0
for f in tqdm(it, total=len(foods_w)):
    num = num+1
    if(num > len(foods_w)): #only allows the loop to run 100 times for now
        break
    else:
        try:
            d = Daily(stat[f[19]], f[0], f[0]).fetch()
            d['BRNCH_CD'] = f[1]
            weather.append(d)
        except:
            weather.append(0) #adds a zero for the observations where weather info is unavailable

#d = Daily()
#d['tavg'][0] #how to access the individual values from the daily info

#TODO have to figure out which weather values (temperature, percipitation, etc... are valuable for us to have in our model)

#Foods3.to_csv('model_data.csv', index=False)

#https://dev.meteostat.net/python/daily.html#api #site with all of the info on the data


Serializing the weather data so i dont have to fetch the weather data agian (this saves the python objects to a file)

In [ ]:
import pickle

#writes all of the weather data to a file 
with open("weather.pickle", "wb") as outfile:
    pickle.dump(weather, outfile)


#with open("weather.pickle", "rb") as infile:
 #   weather1 = pickle.load(infile)


Doing analysis on the weather data 

In [ ]:
import matplotlib.pyplot as plt
import pickle


with open("weather.pickle", "rb") as infile:
    weather1 = pickle.load(infile)




weather1 = weather
weather_pd = pd.concat(weather1)



foods_w1 = foods_w
foods_w1 = foods_w1.reset_index()
weather_pd = pd.concat(weather1)
weather_pd['Date'] = weather_pd.index
weather_pd.index = range(0,len(weather_pd),1)

foods_w1 = pd.merge(foods_w1, weather_pd, on=['BRNCH_CD', 'Date'])


foods_w1.to_csv('weather_data.csv', index=False)


foods_w1.columns



b = pd.DataFrame()
b['tavg'] = foods_w1['tavg']
b['cpmh'] = foods_w1

plt.boxplot()
plt.show()



In [ ]:

""""""""""""""'Preliminary regression testing'

#creating a base linear model with cpmh as the response 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3) 
reg = LinearRegression().fit(x_train, y_train)
pred = reg.predict(x_test)
mean_squared_error(y_test, pred)
r2_score(y_test, pred)


"""""""""""" "Testing Code"
Foods2['weekday'] = Foods2['Date'].apply(lambda x: weekdays[x.weekday()]) #adds the weekday of the particular entry 

x = datetime(2022,12,16)
d = datetime(2023, 1, 8)
l = 0


x.isoweekday()
weekdays[x.weekday()]

type(Foods.Date.iloc[0])
Foods2.columns


""""""""""""'useful code'
cases.where(cases['DIV_NM'] == h)['BRNCH_CD'].dropna(how='all').iloc[0] #line to get the unique branch code for each division name
#code to fetch weather data 
f = foods_w.iloc[1500]
s = station.nearby(f[19][0], f[19][1]).fetch(1)
d = Daily(s, f[0], f[0]).fetch()
d['tavg'][0] #how to access the individual values from the daily info